# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 3 2022 2:11PM,250000,10,DNMD-26011,Emerginnova,Released
1,Nov 3 2022 2:11PM,250000,10,Enova-10927,Emerginnova,Released
2,Nov 3 2022 2:11PM,250000,10,Enova-10928,Emerginnova,Released
3,Nov 3 2022 2:11PM,250000,10,Enova-10929,Emerginnova,Released
4,Nov 3 2022 2:10PM,249998,20,8728953,Alumier Labs Inc.,Released
5,Nov 3 2022 2:09PM,249999,22,632561,"NBTY Global, Inc.",Released
6,Nov 3 2022 2:08PM,249997,10,0086142244,ISDIN Corporation,Released
7,Nov 3 2022 2:08PM,249997,10,0086142256,ISDIN Corporation,Released
8,Nov 3 2022 2:08PM,249997,10,0086142303,ISDIN Corporation,Released
9,Nov 3 2022 2:08PM,249997,10,0086142316,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,249996,Released,1
18,249997,Released,7
19,249998,Released,1
20,249999,Released,1
21,250000,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249996,NaN,NaN,1.0
249997,NaN,NaN,7.0
249998,NaN,NaN,1.0
249999,NaN,NaN,1.0
250000,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249950,37.0,4.0,4.0
249956,0.0,0.0,1.0
249969,0.0,0.0,2.0
249970,4.0,2.0,1.0
249979,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249950,37,4,4
249956,0,0,1
249969,0,0,2
249970,4,2,1
249979,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249950,37,4,4
1,249956,0,0,1
2,249969,0,0,2
3,249970,4,2,1
4,249979,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249950,37,4,4
1,249956,,,1
2,249969,,,2
3,249970,4,2,1
4,249979,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 3 2022 2:11PM,250000,10,Emerginnova
4,Nov 3 2022 2:10PM,249998,20,Alumier Labs Inc.
5,Nov 3 2022 2:09PM,249999,22,"NBTY Global, Inc."
6,Nov 3 2022 2:08PM,249997,10,ISDIN Corporation
13,Nov 3 2022 2:04PM,249994,10,ISDIN Corporation
30,Nov 3 2022 2:03PM,249995,10,Bio-PRF
34,Nov 3 2022 1:59PM,249993,10,ISDIN Corporation
41,Nov 3 2022 1:53PM,249992,20,"HVL, LLC dba Atrium Innovations"
42,Nov 3 2022 1:09PM,249986,16,Sartorius Bioprocess Solutions
43,Nov 3 2022 1:06PM,249996,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 3 2022 2:11PM,250000,10,Emerginnova,,,4
1,Nov 3 2022 2:10PM,249998,20,Alumier Labs Inc.,,,1
2,Nov 3 2022 2:09PM,249999,22,"NBTY Global, Inc.",,,1
3,Nov 3 2022 2:08PM,249997,10,ISDIN Corporation,,,7
4,Nov 3 2022 2:04PM,249994,10,ISDIN Corporation,,,17
5,Nov 3 2022 2:03PM,249995,10,Bio-PRF,,,4
6,Nov 3 2022 1:59PM,249993,10,ISDIN Corporation,,,7
7,Nov 3 2022 1:53PM,249992,20,"HVL, LLC dba Atrium Innovations",,,1
8,Nov 3 2022 1:09PM,249986,16,Sartorius Bioprocess Solutions,,,1
9,Nov 3 2022 1:06PM,249996,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 2:11PM,250000,10,Emerginnova,4,,
1,Nov 3 2022 2:10PM,249998,20,Alumier Labs Inc.,1,,
2,Nov 3 2022 2:09PM,249999,22,"NBTY Global, Inc.",1,,
3,Nov 3 2022 2:08PM,249997,10,ISDIN Corporation,7,,
4,Nov 3 2022 2:04PM,249994,10,ISDIN Corporation,17,,
5,Nov 3 2022 2:03PM,249995,10,Bio-PRF,4,,
6,Nov 3 2022 1:59PM,249993,10,ISDIN Corporation,7,,
7,Nov 3 2022 1:53PM,249992,20,"HVL, LLC dba Atrium Innovations",1,,
8,Nov 3 2022 1:09PM,249986,16,Sartorius Bioprocess Solutions,1,,
9,Nov 3 2022 1:06PM,249996,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 2:11PM,250000,10,Emerginnova,4,,
1,Nov 3 2022 2:10PM,249998,20,Alumier Labs Inc.,1,,
2,Nov 3 2022 2:09PM,249999,22,"NBTY Global, Inc.",1,,
3,Nov 3 2022 2:08PM,249997,10,ISDIN Corporation,7,,
4,Nov 3 2022 2:04PM,249994,10,ISDIN Corporation,17,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 2:11PM,250000,10,Emerginnova,4.0,NaN,NaN
1,Nov 3 2022 2:10PM,249998,20,Alumier Labs Inc.,1.0,NaN,NaN
2,Nov 3 2022 2:09PM,249999,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Nov 3 2022 2:08PM,249997,10,ISDIN Corporation,7.0,NaN,NaN
4,Nov 3 2022 2:04PM,249994,10,ISDIN Corporation,17.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 2:11PM,250000,10,Emerginnova,4.0,0.0,0.0
1,Nov 3 2022 2:10PM,249998,20,Alumier Labs Inc.,1.0,0.0,0.0
2,Nov 3 2022 2:09PM,249999,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Nov 3 2022 2:08PM,249997,10,ISDIN Corporation,7.0,0.0,0.0
4,Nov 3 2022 2:04PM,249994,10,ISDIN Corporation,17.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1749918,42.0,2.0,4.0
15,249950,4.0,4.0,37.0
16,249986,1.0,0.0,0.0
19,749916,3.0,0.0,0.0
20,499990,2.0,0.0,0.0
21,499969,1.0,1.0,0.0
22,499995,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1749918,42.0,2.0,4.0
1,15,249950,4.0,4.0,37.0
2,16,249986,1.0,0.0,0.0
3,19,749916,3.0,0.0,0.0
4,20,499990,2.0,0.0,0.0
5,21,499969,1.0,1.0,0.0
6,22,499995,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42.0,2.0,4.0
1,15,4.0,4.0,37.0
2,16,1.0,0.0,0.0
3,19,3.0,0.0,0.0
4,20,2.0,0.0,0.0
5,21,1.0,1.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,15,Released,4.0
2,16,Released,1.0
3,19,Released,3.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,4.0,37.0,0.0,0.0,0.0,0.0,0.0
Executing,2.0,4.0,0.0,0.0,0.0,1.0,0.0
Released,42.0,4.0,1.0,3.0,2.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,4.0,37.0,0.0,0.0,0.0,0.0,0.0
1,Executing,2.0,4.0,0.0,0.0,0.0,1.0,0.0
2,Released,42.0,4.0,1.0,3.0,2.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,4.0,37.0,0.0,0.0,0.0,0.0,0.0
1,Executing,2.0,4.0,0.0,0.0,0.0,1.0,0.0
2,Released,42.0,4.0,1.0,3.0,2.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()